# Using Python and OWSLib

[OWSLib](https://geopython.github.io/OWSLib) is a Python package which provides Pythonic access to OGC APIs and web services.  Let's see how easy it is to work with wis2box with standards-based tooling:

In [1]:
from owslib.ogcapi.features import Features

import pandas as pd

def pretty_print(input):
    print(json.dumps(input, indent=2))


api = 'http://localhost:8999/oapi'

Let's load the wis2box API into OWSLib and inspect some data

In [2]:
oafeat = Features(api)

collections = oafeat.collections()
print(f'This OGC API Features endpoint has {len(collections["collections"])} datasets')

for dataset in collections['collections']:
    print(dataset['title'])

malawi_obs = oafeat.collection_items('mwi.mwi_met_centre.data.core.weather.surface-based-observations.SYNOP')
malawi_obs_df = pd.DataFrame(malawi_obs['features'])

# then filter by station
obs = oafeat.collection_items('mwi.mwi_met_centre.data.core.weather.surface-based-observations.SYNOP', wigos_station_identifier='0-454-2-AWSCHIDOOLE', name='air_temperature', limit=10000)

datestamp = [obs['properties']['resultTime'] for obs in obs['features']]
air_temperature = [obs['properties']['value'] for obs in obs['features']]

d = {
    'Date/Time': datestamp,
    'Air temperature (°C)': air_temperature
}

df = pd.DataFrame(data=d)

This OGC API Features endpoint has 4 datasets
Surface weather observations (passthrough)
Discovery metadata
Stations
Surface weather observations (hourly)


In [3]:
df.dtypes

Date/Time                object
Air temperature (°C)    float64
dtype: object

In [4]:
df.head(3)

,Date/Time,Air temperature (°C)
0,2022-01-12T13:55:00Z,24.85
1,2022-01-12T14:55:00Z,27.25
2,2022-01-12T15:55:00Z,26.65


In [5]:
print("Time extent\n")
print(f'Begin: {df["Date/Time"].min()}')
print(f'End: {df["Date/Time"].max()}')

print("Summary statistics:\n")
df[['Air temperature (°C)']].describe()

Time extent

Begin: 2022-01-12T13:55:00Z
End: 2022-06-10T14:55:00Z
Summary statistics:



,Air temperature (°C)
count,5106.000000
mean,23.541559
std,4.053172
min,13.550000
25%,20.950000
50%,23.350000
75%,26.350000
max,37.850000
